In [1]:
import numpy as np
from numpy import loadtxt
import fitsio
import healpy as hp
import healsparse as hsp
import skyproj
import matplotlib.pyplot as plt
import Config
from os import listdir
from astropy.table import Table
import astropy.units as u
from astropy.coordinates import SkyCoord

In [2]:
res = 512

Check the number of galaxies that get correctly classified as galaxies. Based on pixel locations, get an idea for what the expectation value for this would be given the galaxy correct classification probabilities that were calculated out. The ratio of these two is a multiplicative factor to be applied to the galaxy correct classification probability map. Same goes for stars.

In [3]:
deepCols  = ['KNN_CLASS', 'RA', 'DEC', 'BDF_MAG_DERED_G', 'BDF_MAG_DERED_R', 'MASK_FLAGS', 'MASK_FLAGS_NIR']
deepRA = np.array([])
deepDEC = np.array([])
deepG = np.array([])
deepGR = np.array([])
deepMasks = np.array([])
deepClass = np.array([])
deepFlag = np.array([])
deepFlagNir = np.array([])

for i in (np.arange(5) + 1):
    deepFile = '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_00000' + str(i) + '.fits'

    deepData = fitsio.read(deepFile, columns = deepCols)

    deepRA = np.append(deepRA, deepData['RA'])
    deepDEC = np.append(deepDEC, deepData['DEC'])
    deepG = np.append(deepG, deepData['BDF_MAG_DERED_G'])
    deepGR = np.append(deepGR, deepData['BDF_MAG_DERED_G'] - deepData['BDF_MAG_DERED_R'])
    deepClass = np.append(deepClass, deepData['KNN_CLASS'])
    deepFlag = np.append(deepFlag, deepData['MASK_FLAGS'])
    deepFlagNir = np.append(deepFlagNir, deepData['MASK_FLAGS_NIR'])

cutIndices = np.where((deepFlag == 0) &
                      (deepFlagNir == 0) &
                      (deepGR <= 1) &
                      (deepGR >= -0.3) &
                      (deepG <= 25) &
                      (deepG > 24) &
                      (deepRA < 120) &
                      (deepClass > 0) &
                      (deepClass < 3))[0]

deepG = deepG[cutIndices]
deepClass = deepClass[cutIndices]
deepRA = deepRA[cutIndices]
deepDEC = deepDEC[cutIndices]

deepPix = np.unique(hp.ang2pix(res, deepRA, deepDEC, lonlat = True, nest = True))

deepPixCheck = np.full(12*(res**2), False, dtype = bool)
deepPixCheck[deepPix] = True

In [4]:
# likeDir = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/MaximumLikelihood/'
# caliDir = likeDir + 'Calibration/'
# minGR = -0.3
# maxGR = 1
# pixFile = likeDir + 'PixAndConds/Valid_'+str(res)+'_Pixels.fits'
# validPix = fitsio.read(pixFile)['PIXEL']
# dirStars = '/hdfs/bechtol/balrog/y3/y3a2_gold_v2p2_skim/healpixel2/'
# files = listdir(dirStars)
# usedCols = ['FLAGS_FOREGROUND', 'FLAGS_BADREGIONS', 'FLAGS_FOOTPRINT', 'EXTENDED_CLASS_SOF', 'SOF_PSF_MAG_G', 'SOF_PSF_MAG_R', 'SOF_CM_MAG_G', 'SOF_CM_MAG_R', 'RA', 'DEC']
# pixCheck = np.full(12*(res**2), False, dtype = bool)
# pixCheck[validPix] = True

# magBins = [[23, 24], [24, 25], [-1000, 20], [20, 22], [22, 23], [23, 24], [24, 25]]
# classCutoffs = [2.5, 2.5, 0.5, 0.5, 0.5, 0.5, 0.5]
# starPosFiles = [caliDir + 'Class_2.5_Star_Pos_23-24.fits', caliDir + 'Class_2.5_Star_Pos_24-25.fits', caliDir + 'Class_0.5_Star_Pos_<20.fits', caliDir + 'Class_0.5_Star_Pos_20-22.fits', caliDir + 'Class_0.5_Star_Pos_22-23.fits', caliDir + 'Class_0.5_Star_Pos_23-24.fits', caliDir + 'Class_0.5_Star_Pos_24-25.fits']
# galaPosFiles = [caliDir + 'Class_2.5_Gala_Pos_23-24.fits', caliDir + 'Class_2.5_Gala_Pos_24-25.fits', caliDir + 'Class_0.5_Gala_Pos_<20.fits', caliDir + 'Class_0.5_Gala_Pos_20-22.fits', caliDir + 'Class_0.5_Gala_Pos_22-23.fits', caliDir + 'Class_0.5_Gala_Pos_23-24.fits', caliDir + 'Class_0.5_Gala_Pos_24-25.fits']

# for i in np.arange(len(classCutoffs)):
    
#     magBin = magBins[i]
#     classCutoff = classCutoffs[i]
#     starPosFile = starPosFiles[i]
#     galaPosFile = galaPosFiles[i]
    
#     StarRA = []
#     StarDEC = []

#     GalaRA = []
#     GalaDEC = []

#     for file in files:
#         obsData = fitsio.read(dirStars + file, columns = usedCols)
#         FOREGROUND = obsData[usedCols[0]]
#         BADREGIONS = obsData[usedCols[1]]
#         FOOTPRINT = obsData[usedCols[2]]
#         CLASS = obsData[usedCols[3]]
#         GMAG = obsData[usedCols[4]]
#         RMAG = obsData[usedCols[5]]
#         GMAG_GALA = obsData[usedCols[6]]
#         RMAG_GALA = obsData[usedCols[7]]
#         RA = obsData[usedCols[8]]
#         DEC = obsData[usedCols[9]]

#         magCut = np.where((GMAG <= magBin[1]) & (GMAG > magBin[0]))[0]

#         boolMagCut = np.zeros(len(GMAG))
#         boolMagCut[magCut] = 1

#         galaMagCut = np.where((GMAG_GALA <= magBin[1]) & (GMAG_GALA > magBin[0]))[0]

#         boolGalaMagCut = np.zeros(len(GMAG_GALA))
#         boolGalaMagCut[galaMagCut] = 1

#         # Observed Star Objects

#         blueStarCut = np.where((FOREGROUND == 0) & 
#                           (BADREGIONS < 2) & 
#                           (FOOTPRINT == 1) & 
#                           (CLASS <= classCutoff) & 
#                           (CLASS >= 0) &
#                           (boolMagCut > 0) &
#                           ((GMAG - RMAG) <= maxGR) &
#                           ((GMAG - RMAG) >= minGR))[0]

#         STARRA = RA[blueStarCut]
#         STARDEC = DEC[blueStarCut]

#         STARPIX = hp.ang2pix(res, STARRA, STARDEC, lonlat = True, nest = True)

#         STARRA = STARRA[np.where((pixCheck[STARPIX]) & (deepPixCheck[STARPIX]))[0]]
#         STARDEC = STARDEC[np.where((pixCheck[STARPIX]) & (deepPixCheck[STARPIX]))[0]]

#         StarRA.extend(STARRA)
#         StarDEC.extend(STARDEC)

#         # Observed Galaxy Objects

#         blueGalaCut = np.where((FOREGROUND == 0) & 
#                           (BADREGIONS < 2) & 
#                           (FOOTPRINT == 1) & 
#                           (CLASS <= 3) & 
#                           (CLASS >= classCutoff) &
#                           (boolGalaMagCut > 0) &
#                           ((GMAG_GALA - RMAG_GALA) <= maxGR) &
#                           ((GMAG_GALA - RMAG_GALA) >= minGR))[0]

#         GALARA = RA[blueGalaCut]
#         GALADEC = DEC[blueGalaCut]

#         GALAPIX = hp.ang2pix(res, GALARA, GALADEC, lonlat = True, nest = True)

#         GALARA = GALARA[np.where((pixCheck[GALAPIX]) & (deepPixCheck[GALAPIX]))[0]]
#         GALADEC = GALADEC[np.where((pixCheck[GALAPIX]) & (deepPixCheck[GALAPIX]))[0]]

#         GalaRA.extend(GALARA)
#         GalaDEC.extend(GALADEC)

#     StarRA = np.array(StarRA, dtype = object)
#     StarDEC = np.array(StarDEC, dtype = object)

#     GalaRA = np.array(GalaRA, dtype = object)
#     GalaDEC = np.array(GalaDEC, dtype = object)
    
#     my_table = Table()
#     my_table['RA'] = StarRA.astype(float)
#     my_table['DEC'] = StarDEC.astype(float)
#     my_table.write(starPosFile, overwrite = True)
    
#     my_table = Table()
#     my_table['RA'] = GalaRA.astype(float)
#     my_table['DEC'] = GalaDEC.astype(float)
#     my_table.write(galaPosFile, overwrite = True)

In [15]:
likeDir = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/MaximumLikelihood/'
minGR = -0.3
maxGR = 1
magBin = [22, 23]
pixFile = likeDir + 'PixAndConds/Valid_'+str(res)+'_Pixels.fits'
validPix = fitsio.read(pixFile)['PIXEL']
dirStars = '/hdfs/bechtol/balrog/y3/y3a2_gold_v2p2_skim/healpixel2/'
files = listdir(dirStars)
usedCols = ['FLAGS_FOREGROUND', 'FLAGS_BADREGIONS', 'FLAGS_FOOTPRINT', 'EXTENDED_CLASS_SOF', 'SOF_PSF_MAG_G', 'SOF_PSF_MAG_R', 'SOF_CM_MAG_G', 'SOF_CM_MAG_R', 'RA', 'DEC']
pixCheck = np.full(12*(res**2), False, dtype = bool)
pixCheck[validPix] = True

In [16]:
StarRA = []
StarDEC = []

GalaRA = []
GalaDEC = []

for file in files:
    obsData = fitsio.read(dirStars + file, columns = usedCols)
    FOREGROUND = obsData[usedCols[0]]
    BADREGIONS = obsData[usedCols[1]]
    FOOTPRINT = obsData[usedCols[2]]
    CLASS = obsData[usedCols[3]]
    GMAG = obsData[usedCols[4]]
    RMAG = obsData[usedCols[5]]
    GMAG_GALA = obsData[usedCols[6]]
    RMAG_GALA = obsData[usedCols[7]]
    RA = obsData[usedCols[8]]
    DEC = obsData[usedCols[9]]

    magCut = np.where((GMAG <= magBin[1]) & (GMAG > magBin[0]))[0]

    boolMagCut = np.zeros(len(GMAG))
    boolMagCut[magCut] = 1
    
    galaMagCut = np.where((GMAG_GALA <= magBin[1]) & (GMAG_GALA > magBin[0]))[0]
    
    boolGalaMagCut = np.zeros(len(GMAG_GALA))
    boolGalaMagCut[galaMagCut] = 1

    # Observed Star Objects

    blueStarCut = np.where((FOREGROUND == 0) & 
                      (BADREGIONS < 2) & 
                      (FOOTPRINT == 1) & 
                      (CLASS <= 2) & 
                      (CLASS >= 0) &
                      (boolMagCut > 0) &
                      ((GMAG - RMAG) <= maxGR) &
                      ((GMAG - RMAG) >= minGR))[0]

    STARRA = RA[blueStarCut]
    STARDEC = DEC[blueStarCut]
    
    STARPIX = hp.ang2pix(res, STARRA, STARDEC, lonlat = True, nest = True)

    STARRA = STARRA[np.where((pixCheck[STARPIX]) & (deepPixCheck[STARPIX]))[0]]
    STARDEC = STARDEC[np.where((pixCheck[STARPIX]) & (deepPixCheck[STARPIX]))[0]]

    StarRA.extend(STARRA)
    StarDEC.extend(STARDEC)

    # Observed Galaxy Objects

    blueGalaCut = np.where((FOREGROUND == 0) & 
                      (BADREGIONS < 2) & 
                      (FOOTPRINT == 1) & 
                      (CLASS <= 3) & 
                      (CLASS >= 3) &
                      (boolGalaMagCut > 0) &
                      ((GMAG_GALA - RMAG_GALA) <= maxGR) &
                      ((GMAG_GALA - RMAG_GALA) >= minGR))[0]

    GALARA = RA[blueGalaCut]
    GALADEC = DEC[blueGalaCut]

    GALAPIX = hp.ang2pix(res, GALARA, GALADEC, lonlat = True, nest = True)

    GALARA = GALARA[np.where((pixCheck[GALAPIX]) & (deepPixCheck[GALAPIX]))[0]]
    GALADEC = GALADEC[np.where((pixCheck[GALAPIX]) & (deepPixCheck[GALAPIX]))[0]]

    GalaRA.extend(GALARA)
    GalaDEC.extend(GALADEC)
    
StarRA = np.array(StarRA, dtype = object)
StarDEC = np.array(StarDEC, dtype = object)

GalaRA = np.array(GalaRA, dtype = object)
GalaDEC = np.array(GalaDEC, dtype = object)

In [17]:
caliDir = likeDir + 'Calibration/'

starPosFile = caliDir + 'Class_2.5_Star_Pos_22-23.fits'
galaPosFile = caliDir + 'Class_2.5_Gala_Pos_22-23.fits'

In [18]:
my_table = Table()
my_table['RA'] = StarRA.astype(float)
my_table['DEC'] = StarDEC.astype(float)
my_table.write(starPosFile, overwrite = True)

In [19]:
my_table = Table()
my_table['RA'] = GalaRA.astype(float)
my_table['DEC'] = GalaDEC.astype(float)
my_table.write(galaPosFile, overwrite = True)

In [14]:
def findMatches(angleCutoff, RASource, DECSource, RAMatchCatalog, DECMatchCatalog, nthneighbor=1):
    c = SkyCoord(ra=RASource*u.degree, dec=DECSource*u.degree)
    catalog = SkyCoord(ra=RAMatchCatalog*u.degree, dec=DECMatchCatalog*u.degree)
    idx, d2d, d3d = c.match_to_catalog_sky(catalog, nthneighbor=nthneighbor)
    matches = d2d < angleCutoff
    return matches, d2d

In [15]:
allStarData = fitsio.read(starPosFile)
allStarRA = allStarData['RA']
allStarDEC = allStarData['DEC']

allGalaData = fitsio.read(galaPosFile)
allGalaRA = allGalaData['RA']
allGalaDEC = allGalaData['DEC']

In [16]:
# matchedStars, _ = findMatches(0.5*u.arcsec, allStarRA, allStarDEC, deepRA, deepDEC)
# matchedStarRA = allStarRA[matchedStars]
# matchedStarDEC = allStarDEC[matchedStars]

# matchedGalas, _ = findMatches(0.5*u.arcsec, allGalaRA, allGalaDEC, deepRA, deepDEC)
# matchedGalaRA = allGalaRA[matchedGalas]
# matchedGalaDEC = allGalaDEC[matchedGalas]

In [17]:
# doubleGalas, _ = findMatches(0.5*u.arcsec, matchedGalaRA, matchedGalaDEC, matchedGalaRA, matchedGalaDEC, nthneighbor = 2)
# matchedGalaRA = matchedGalaRA[~doubleGalas]
# matchedGalaDEC = matchedGalaDEC[~doubleGalas]

In [18]:
deepStarMatches, _ = findMatches(0.5*u.arcsec, deepRA, deepDEC, allStarRA, allStarDEC)
deepGalaMatches, _ = findMatches(0.5*u.arcsec, deepRA, deepDEC, allGalaRA, allGalaDEC)

matchedDeepStarRA = deepRA[deepStarMatches]
matchedDeepStarDEC = deepDEC[deepStarMatches]
matchedDeepStarClass = deepClass[deepStarMatches]

matchedDeepGalaRA = deepRA[deepGalaMatches]
matchedDeepGalaDEC = deepDEC[deepGalaMatches]
matchedDeepGalaClass = deepClass[deepGalaMatches]

# if len(matchedDeepRA) != (len(matchedGalaRA) + len(matchedStarRA)):
#     raise Exception('Mismatch in objects')

In [19]:
# TS = True Star, FS = False Star (misclassified galaxy as a star), same for galaxy

TSPIX = hp.ang2pix(res, matchedDeepStarRA[np.where(matchedDeepStarClass == 2)[0]], matchedDeepStarDEC[np.where(matchedDeepStarClass == 2)[0]], lonlat = True, nest = True)
FSPIX = hp.ang2pix(res, matchedDeepStarRA[np.where(matchedDeepStarClass == 1)[0]], matchedDeepStarDEC[np.where(matchedDeepStarClass == 1)[0]], lonlat = True, nest = True)

TGPIX = hp.ang2pix(res, matchedDeepGalaRA[np.where(matchedDeepGalaClass == 1)[0]], matchedDeepGalaDEC[np.where(matchedDeepGalaClass == 1)[0]], lonlat = True, nest = True)
FGPIX = hp.ang2pix(res, matchedDeepGalaRA[np.where(matchedDeepGalaClass == 2)[0]], matchedDeepGalaDEC[np.where(matchedDeepGalaClass == 2)[0]], lonlat = True, nest = True)

In [20]:
len(TSPIX) / (len(TSPIX) + len(FGPIX))

0.3895809739524349

In [21]:
len(TGPIX) / (len(TGPIX) + len(FSPIX))

0.9388040455512918

In [22]:
# TGPIX

In [23]:
# test = np.full(12*(res**2), hp.UNSEEN)
# test[validPix] = 0
# test[TSPIX] = 1
# nside_coverage = 32
# hspTest = hsp.HealSparseMap(nside_coverage=nside_coverage, healpix_map=test)

In [24]:
# # The default DES projection is a McBrydeSkymap.
# fig = plt.figure(1, figsize=(8, 6))
# ax = fig.add_subplot(111)
# sp = skyproj.DESSkyproj(ax=ax)
# sp.draw_hspmap(hspTest, cmap = 'bwr')
# plt.clim(0, 1)
# plt.colorbar(location = 'bottom', label = 'Pixels With Matches')
# fig.suptitle('Locations of Deep Field Matches')
# plt.show()

In [25]:
starDir = likeDir + 'Stars/'

starProbFile = starDir + 'Star_Prob_24-25.fits'

In [26]:
galaDir = likeDir + 'Galaxies/'

galaProbFile = galaDir + 'Gala_Prob_24-25.fits'

In [27]:
starCorrProb = np.clip(fitsio.read(starProbFile)['SIGNAL'], 0, 1)
fullStarProb = np.full(12*(res**2), hp.UNSEEN)
fullStarProb[validPix] = starCorrProb

galaCorrProb = np.clip(fitsio.read(galaProbFile)['SIGNAL'], 0, 1)
fullGalaProb = np.full(12*(res**2), hp.UNSEEN)
fullGalaProb[validPix] = galaCorrProb

In [28]:
len(TSPIX) / (np.sum(fullStarProb[TSPIX[np.where(pixCheck[TSPIX])[0]]]) + np.sum(fullStarProb[FGPIX[np.where(pixCheck[FGPIX])[0]]]))

0.8106019195650382

In [29]:
len(TGPIX) / (np.sum(fullGalaProb[TGPIX[np.where(pixCheck[TGPIX])[0]]]) + np.sum(fullGalaProb[FSPIX[np.where(pixCheck[FSPIX])[0]]]))

1.0257101791939418